In [1]:
import os

In [2]:
%pwd

'f:\\datascienceproject\\research'

In [4]:
os.chdir("../")
%pwd

'f:\\datascienceproject'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml,create_directories

In [12]:
class ConfigurationManager:
    def __init__(self,
                config_filepath=CONFIG_FILE_PATH,
                params_filepath=PARAMS_FILE_PATH,
                schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [13]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile

In [18]:
## component-Data Ingestion

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    # download zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
             zip_ref.extractall(unzip_path)

       

In [20]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2025-11-18 17:47:58,894: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-18 17:47:58,926: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-18 17:47:58,931: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-11-18 17:47:59,289: INFO: common: created directory at: artifacts]
[2025-11-18 17:47:59,289: INFO: common: created directory at: artifacts/data_ingestion]
[2025-11-18 17:48:02,608: INFO: 3251832804: artifacts/data_ingestion/winequality-red.csv download! with following info: 
Connection: close
Content-Length: 100951
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: text/plain; charset=utf-8
ETag: "3e81938066ce4b085d54bdbd15389504f01d1066303c384c703ddc5322529525"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6876:6FA9D:258332:47E5E6:691C63FA
Accept-Ranges

BadZipFile: File is not a zip file